In [ ]:
#A ordem da sequência do QKD EnQuad é a seguinte:
#1- Polarizer
#2- Eve’s Intercept and Resend
#3- Channel
#4- Beam Splitter
#5- Polarizing beam splitter
#6- SPAD
#7- Sifting
#8- Security Testing and Recommendations

#Entradas:
#Número de pulsos (fótons) a serem enviados. Não há limite; pode ser 100, 10.000 ou mais. No entanto, o tempo de processamento aumenta diretamente.
#Nível de ataque: mínimo: 0, máximo: 1.
#Probabilidade de despolarização: mínimo: 0, máximo: 0.25.
#Os bits da chave podem ser gerados da seguinte forma: key_bits = np.round(np.random.rand(1, number_of_photons))
#A seleção de base de Alice pode ser gerada usando um gerador binário aleatório da seguinte forma: Alice_basis_selection = np.round(np.random.rand(1, number_of_photons))
#onde a base retangular ((+) base) é representada pelo símbolo "0" e a base circular ((X) base) é representada pelo símbolo "1".
#A seleção de base de Bob pode ser gerada da mesma forma que a de Alice.

#Principais Saídas:
#Chave reconciliada: Chave secreta após a reconciliação.
#Taxa de bits da chave secreta: Razão entre o comprimento da chave reconciliada e o comprimento da chave filtrada.
#Tempo de simulação do EnQuad.

import numpy as np
import time
from Polarizer import Polarizer
from Eve_Intercept_and_Resend import Eve_Intercept_and_Resend
from Security_Testing_and_Reconciliation_Efficiency_Reporting import Security_Testing_and_Reconciliation_Efficiency_Reporting
from Channel import Channel
from Polarizing_Beam_Splitter import Polarizing_Beam_Splitter
from Beam_Splitter import Beam_Splitter
from SPAD import SPAD
from Sifting import Sifting

# Exemplo das entradas
print("---------------------------------------------------------------------------------")
print("-----<<EnQuad Engine Initialization>>-----\n")
print("-----<<Input Conditions>>------\n")

# Entrada de número de fótons
number_of_photons = int(input('Please input the number of photons to be sent:  '))
while True:
    if number_of_photons > 0 and (number_of_photons % 1) == 0:
        break
    else:
        number_of_photons = int(input('Number of photons must be an integer greater than zero: '))

# Entrada de probabilidade de despolarização do canal
channel_depolarization_probability = float(input('Please input the depolarizing parameter of your channel:  '))
while True:
    if 0 < channel_depolarization_probability < 0.25:
        break
    else:
        channel_depolarization_probability = float(input('Depolarizing parameter should be a decimal number in the range of ]0 1/4[: '))

# Nível de ataque de Eve
Eve_attack_level = float(input('Please input Eve attack level of your interest:   '))
while True:
    if 0 <= Eve_attack_level <= 1:
        break
    else:
        Eve_attack_level = float(input('Eve attack Level should be a decimal number in the range of [0 1]: '))

# Geração de bits de chave e seleção de bases de Alice e Bob
key_bits = np.round(np.random.rand(number_of_photons))
Alice_basis_selection = np.round(np.random.rand(number_of_photons))
Bob_basis_selection = np.round(np.random.rand(number_of_photons))

print("-----<<Phases>>------\n")

# 1 - Polarizador
start_time = time.time()
Alice_polarized_photons_states = Polarizer(Alice_basis_selection, key_bits, number_of_photons)
t1 = time.time() - start_time
print('Polarizer phase...Done')

copy_before_eve = Alice_polarized_photons_states

# 2 - Interceptação e Reenvio de Eve
t2 = 0
if Eve_attack_level != 0:
    start_time = time.time()
    Alice_polarized_photons_states, Eve_basis_selection = Eve_Intercept_and_Resend(Eve_attack_level, number_of_photons, Alice_polarized_photons_states)
    Eve_key = SPAD(Alice_polarized_photons_states)
    t2 = time.time() - start_time
    print('Eve-Intercept-And-Resend phase...Done')
else:
    print('Eve: None')

# 3 - Canal
start_time = time.time()
Alice_polarized_photons_states_after_channel = Channel(number_of_photons, channel_depolarization_probability, Alice_polarized_photons_states)
t3 = time.time() - start_time
print('Channel phase...Done')

# 4 - Divisor de feixe
start_time = time.time()
photon_states_after_beam_splitter = Beam_Splitter(Alice_polarized_photons_states_after_channel, Bob_basis_selection)
t4 = time.time() - start_time
print('Beam Splitter phase...Done')

# 5 - Divisor de feixe polarizador
start_time = time.time()
photons_states_after_polarizing_beam_splitter = Polarizing_Beam_Splitter(photon_states_after_beam_splitter, number_of_photons)
t5 = time.time() - start_time
print('Polarizing Beam Splitter...Done')

# 6 - Diodo de Avalanche de Fótons Únicos (SPAD)
start_time = time.time()
measured_bits = SPAD(photons_states_after_polarizing_beam_splitter)
t6 = time.time() - start_time
print('Single-Photon Avalanche Diode phase...Done')

# 7 - Peneirar
start_time = time.time()
QBER, positions_of_unmatched_bases, Alice_sifted_key, Bob_sifted_key = Sifting(key_bits, Alice_basis_selection, Bob_basis_selection, measured_bits)
t7 = time.time() - start_time
print('Sifting phase...Done')
print(f'Quantum Bit Error Rate is {QBER:.6f}.')

# Tempo total até a filtragem
total_elapsed_time = t1 + t2 + t3 + t4 + t5 + t6 + t7
print(f'EnQuad simulation time taken to compute the sifted key is {total_elapsed_time:.3f} secs.')

# Teste de segurança e relatório de eficiência de reconciliação
Security_Testing_and_Reconciliation_Efficiency_Reporting(number_of_photons, Alice_sifted_key, channel_depolarization_probability, Eve_attack_level)    

---------------------------------------------------------------------------------
-----<<EnQuad Engine Initialization>>-----

-----<<Input Conditions>>------

